# Lab 1: 個人予算アシスタント Strands エージェントの開発

## 概要

この Lab では、Strands Agents を使用して洗練された個人予算アシスタントを作成します。基本的な会話エージェントから始め、モデル設定、会話管理、カスタムツール、構造化出力などの高度な機能を段階的に追加していきます。

この Lab では、実践的な実装を通じて Strands Agents のコアコンセプトを示し、各機能がどのように前の機能の上に構築されて包括的な財務アドバイザリーシステムを作成するかを説明します。最終的には、パーソナライズされた予算アドバイス、支出分析、財務推奨を提供できる本番環境対応のエージェントが完成します。

私たちは、インテリジェントな会話と専門ツールを通じてユーザーの個人財務管理を支援する包括的な**予算エージェント**を作成しています。エージェントは予算ガイダンス、支出パターンの分析、実行可能な財務アドバイスを提供します。

![architecture](./images/single-agent.png)

### 予算エージェントのツールと機能

| ツール | 説明 | 使用例 |
|------|-------------|------------------|
| **calculate_budget** | 月収に基づいて 50/30/20 予算内訳を計算 | 「月収5000ドルで予算を作成してください」 |
| **create_financial_chart** | 財務データの円グラフと可視化を生成 | 「カテゴリ別の支出を可視化してください」 |
| **calculator** | 財務計画のための数学計算を実行 | 「月収の15%を貯蓄用に計算してください」 |

### エージェント機能の概要

予算エージェントには以下が含まれます:

- **パーソナライズされた財務ガイダンス**: 収入と支出パターンに基づいたカスタマイズされたアドバイス
- **インタラクティブな予算作成**: 実績のある 50/30/20 ルールを使用したリアルタイム予算計算
- **ビジュアル分析**: 財務データをより理解しやすくするためのチャート生成
- **会話メモリ**: パーソナライズされた体験のための複数のやり取りにわたるコンテキスト保持
- **構造化されたレポート**: 健全性スコアと推奨事項を含む一貫した解析可能な財務レポート
- **責任ある AI**: 倫理的な財務アドバイスのための組み込みガードレールと免責事項

エージェントは予算作成と支出分析に特化し、投資アドバイスなしで実用的で実行可能なガイダンスを提供します。後続の Lab で構築するより複雑なマルチエージェントシステムの基盤となります。

In [ ]:
# Strands エージェントとツールに必要な依存関係をインストール
!pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
# 予算エージェント用の Strands コアコンポーネントとユーティリティをインポート
from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import calculator
from utils import create_guardrail, pretty_print_messages
import time
import matplotlib.pyplot as plt

### Amazon Bedrock Guardrail と Strands の関連付け

Amazon Bedrock は Strands Agents SDK と直接統合する[組み込みガードレールフレームワーク](https://docs.aws.amazon.com/bedrock/latest/userguide/guardrails.html)を提供しています。ガードレールがトリガーされると、Strands Agents SDK は会話履歴内のユーザー入力を自動的に上書きします。これは、後続の質問が同じ質問によってブロックされないようにするためです。これは `guardrail_redact_input` ブール値で設定でき、`guardrail_redact_input_message` 文字列で上書きメッセージを変更できます。さらに、モデルの出力にも同じ機能が組み込まれていますが、これはデフォルトで無効になっています。`guardrail_redact_output` ブール値でこれを有効にし、`guardrail_redact_output_message` 文字列で上書きメッセージを変更できます。

In [ ]:
# コンテンツフィルタリングと安全性のための Bedrock ガードレールを作成
guardrail_id, guardrail_arn = create_guardrail()

![guardrail](./images/guardrail.png)

以下は、コードで Bedrock ガードレールを活用する方法の例です:

**Amazon Bedrock で Anthropic Claude 3.7 Sonnet の[モデルアクセス](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access-modify.html)を有効にしていることを確認してください。**

In [ ]:
# Claude 3.7 Sonnet とガードレール付きで Bedrock モデルを設定
bedrock_model = BedrockModel(
    model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    region_name="us-west-2",
    temperature=0.0,  # 財務アドバイスのための決定論的な応答
    guardrail_id=guardrail_id,  # Bedrock ガードレール ID
    guardrail_version="DRAFT",  # ガードレールバージョン
    guardrail_trace="enabled",
)

In [ ]:
# 設定済みの Bedrock モデルで基本エージェントを作成
agent = Agent(model=bedrock_model)

In [ ]:
# 一般的な質問で基本エージェント機能をテスト
response_1 = agent("こんにちは！何ができますか？")

In [ ]:
# 投資アドバイスをブロックするガードレールをテスト（フィルタリングされるはず）
response_2 = agent("ビットコインへの投資アドバイスをください")

In [ ]:
# フォーマット付きで会話履歴を表示
pretty_print_messages(messages=agent.messages)

## 予算エージェントの作成

### ステップ 1: システムプロンプトの定義

In [ ]:
# 予算に特化した財務アシスタント用のシステムプロンプトを定義
BUDGET_SYSTEM_PROMPT = """You are a helpful personal finance assistant. 
You provide general strategies for creating budgets, tips on financial discipline to achieve financial milestones, and analyze financial trends. 
You do not provide any investment advice. Keep responses concise and actionable. Always provide 2-3 specific steps the user can take. Focus on practical budgeting and spending advice.
"""

In [ ]:
# カスタムシステムプロンプトで予算エージェントを作成
budget_agent_sys = Agent(
    model=bedrock_model, system_prompt=BUDGET_SYSTEM_PROMPT  # システムプロンプトを関連付け
)

In [ ]:
# 外食支出分析で予算エージェントをテスト
response_3 = budget_agent_sys(
    "月収5000ドルの人が外食に月800ドル使っています。これは多すぎますか？"
)

### ステップ 2: 会話マネージャーの追加

Strands Agents SDK において、コンテキストとはエージェントが理解と推論のために提供される情報を指します。これには以下が含まれます:

- ユーザーメッセージ
- エージェントの応答
- ツールの使用と結果
- システムプロンプト

会話が長くなるにつれて、このコンテキストの管理がいくつかの理由でますます重要になります:

- **トークン制限**: 言語モデルには固定のコンテキストウィンドウ（処理できる最大トークン数）があります
- **パフォーマンス**: より大きなコンテキストにはより多くの処理時間とリソースが必要です
- **関連性**: 古いメッセージは現在の会話にとって関連性が低くなる可能性があります
- **一貫性**: 論理的な流れを維持し、重要な情報を保持する必要があります

#### 会話マネージャーの種類

Strands Agents は、異なるコンテキスト管理ニーズに対応する3種類の会話マネージャーを提供しています:

1. **[SlidingWindowConversationManager](https://strandsagents.com/latest/documentation/docs/api-reference/agent/#strands.agent.conversation_manager.sliding_window_conversation_manager.SlidingWindowConversationManager)**（デフォルト）: 最近のメッセージペアの固定数を維持し、制限に達すると最も古いものを自動的に削除するスライディングウィンドウ戦略を実装します。これは Agent クラスで使用されるデフォルトの会話マネージャーであり、最新のコンテキストが最も重要なほとんどのアプリケーションに最適です。

2. **[NullConversationManager](https://strandsagents.com/latest/documentation/docs/api-reference/agent/#strands.agent.conversation_manager.null_conversation_manager.NullConversationManager)**: 会話履歴を変更しないシンプルな実装です。コンテキスト制限を超えない短い会話、デバッグ目的、またはコンテキストを手動で管理したい場合に役立ちます。

3. **[SummarizingConversationManager](https://strandsagents.com/latest/documentation/docs/api-reference/agent/#strands.agent.conversation_manager.summarizing_conversation_manager.SummarizingConversationManager)**: 古いメッセージを単に破棄するのではなく要約することで、インテリジェントな会話コンテキスト管理を実装します。このアプローチはコンテキスト制限内に収まりながら重要な情報を保持するため、履歴コンテキストが重要な長時間実行される会話に最適です。

In [ ]:
# コンテキスト処理用の会話マネージャーをインポート
from strands.agent.conversation_manager import SummarizingConversationManager

In [ ]:
# 長い会話を処理するための会話マネージャーを設定
conversation_manager = SummarizingConversationManager(
    summary_ratio=0.5,  # コンテキスト削減が必要な場合、メッセージの50%を要約
    preserve_recent_messages=3,  # 常に直近3件のメッセージを保持
)

In [ ]:
# 会話管理機能付きのエージェントを作成
budget_agent_manager = Agent(
    model=bedrock_model,
    system_prompt=BUDGET_SYSTEM_PROMPT,
    conversation_manager=conversation_manager,  # 会話マネージャーを関連付け
    callback_handler=None,
)

### ステップ 3: ストリーミング用の非同期イテレーター

Strands Agents SDK は [stream_async](https://strandsagents.com/latest/documentation/docs/api-reference/agent/#strands.agent.agent.Agent.stream_async) メソッドを通じて非同期イテレーターをサポートしており、Web サーバー、API、その他の非同期アプリケーションなどの非同期環境でエージェントの応答をリアルタイムでストリーミングできます。

In [ ]:
# 非同期イテレーターでストリーミング応答をデモンストレーション
async for event in budget_agent_manager.stream_async(
    "月収5000ドルで外食に800ドル使っています。これは多すぎますか？"
):
    if "data" in event:
        # テキストチャンクのみをクライアントにストリーミング
        print(event["data"], end="")
        time.sleep(0.1) # ストリーミングを適切に表示

### ステップ 4: 財務ツールの追加

In [ ]:
# 50/30/20 予算計算用のカスタムツールを定義
@tool
def calculate_budget(monthly_income: float) -> str:
    """指定された月収に対して 50/30/20 予算内訳を計算します。"""
    needs = monthly_income * 0.50
    wants = monthly_income * 0.30
    savings = monthly_income * 0.20
    return f"💰 ${monthly_income:,.0f}/月の予算:\n• 必需品: ${needs:,.0f} (50%)\n• 欲しいもの: ${wants:,.0f} (30%)\n• 貯蓄: ${savings:,.0f} (20%)"

In [ ]:
# 財務円グラフ作成用のツールを定義
@tool
def create_financial_chart(
    data_dict: dict, chart_title: str = "Financial Chart"
) -> str:
    """財務データの辞書から円グラフの可視化を作成します。"""
    if not data_dict:
        return "❌ チャート用のデータが提供されていません"

    labels = list(data_dict.keys())
    values = list(data_dict.values())
    colors = ["#FF6B6B", "#4ECDC4", "#45B7D1", "#96CEB4", "#FECA57", "#FF9FF3"]

    plt.figure(figsize=(8, 6))
    plt.pie(
        values,
        labels=labels,
        autopct="%1.1f%%",
        colors=colors[: len(values)],
        startangle=90,
    )
    plt.title(f"📊 {chart_title}", fontsize=14, fontweight="bold")
    plt.axis("equal")
    plt.tight_layout()
    plt.show()

    return f"✅ {chart_title} の可視化を作成しました！"

In [ ]:
# すべてのツールを統合した完全な予算エージェントを作成
budget_agent = Agent(
    model=bedrock_model,
    system_prompt=BUDGET_SYSTEM_PROMPT,
    conversation_manager=conversation_manager,
    tools=[calculate_budget, create_financial_chart, calculator],
    callback_handler=None
)

In [ ]:
# ストリーミング応答でツール対応エージェントをテスト
async for event in budget_agent.stream_async(
    "月収5000ドルで外食に800ドル使っています。これは多すぎますか？"
):
    if "data" in event:
        # テキストチャンクのみをクライアントにストリーミング
        print(event["data"], end="")
        time.sleep(0.1) # ストリーミングを適切に表示

### ステップ 5: 財務レポート用の構造化出力の追加

In [ ]:
# 構造化出力モデル用に Pydantic をインポート
from pydantic import BaseModel, Field
from typing import List

In [ ]:
# 構造化財務レポート用の Pydantic モデルを定義
class BudgetCategory(BaseModel):
    name: str = Field(description="予算カテゴリ名")
    amount: float = Field(description="このカテゴリのドル金額")
    percentage: float = Field(description="総収入に対する割合")


class FinancialReport(BaseModel):
    monthly_income: float = Field(description="月間総収入")
    budget_categories: List[BudgetCategory] = Field(
        description="予算カテゴリのリスト"
    )
    recommendations: List[str] = Field(description="具体的な推奨事項のリスト")
    financial_health_score: int = Field(
        ge=1, le=10, description="1〜10の財務健全性スコア"
    )

In [ ]:
# Pydantic モデルを使用して構造化財務レポートを生成
structured_response = budget_agent.structured_output(
    output_model=FinancialReport,
    prompt="月収6000ドルで外食に800ドル使っている人の包括的な財務レポートを生成してください。",
)

In [ ]:
# 構造化レポート出力をフォーマットして表示
print(f"収入: ${structured_response.monthly_income:,.0f}")
for category in structured_response.budget_categories:
    print(f"• {category.name}: ${category.amount:,.0f} ({category.percentage:.1f}%)")
print(f"\n財務健全性スコア: {structured_response.financial_health_score}/10")
print("\n推奨事項:")
for i, rec in enumerate(structured_response.recommendations, 1):
    print(f"{i}. {rec}")

In [ ]:
%%writefile budget_agent.py
# 完全な予算エージェント実装を Python ファイルにエクスポート
from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import calculator
from pydantic import BaseModel, Field
from typing import List
import matplotlib.pyplot as plt


# 財務データ用の構造化出力モデルを定義
class BudgetCategory(BaseModel):
    """予算カテゴリを表すモデル"""
    name: str = Field(description="予算カテゴリ名")
    amount: float = Field(description="このカテゴリのドル金額")
    percentage: float = Field(description="総収入に対する割合")


class FinancialReport(BaseModel):
    """構造化財務レポートを表すモデル"""
    monthly_income: float = Field(description="月間総収入")
    budget_categories: List[BudgetCategory] = Field(
        description="予算カテゴリのリスト"
    )
    recommendations: List[str] = Field(description="具体的な推奨事項のリスト")
    financial_health_score: int = Field(
        ge=1, le=10, description="1〜10の財務健全性スコア"
    )


# 構造化出力用の強化されたシステムプロンプト
BUDGET_SYSTEM_PROMPT = """You are a helpful personal finance assistant. 
You provide general strategies for creating budgets, tips on financial discipline to achieve financial milestones, and analyze financial trends. You do not provide any investment advice. 

When generating financial reports, always provide:
1. Clear budget breakdowns using the 50/30/20 rule or custom allocations
2. Specific, actionable recommendations (2-3 steps)
3. A financial health score based on spending patterns
4. Practical budgeting and spending advice

Use structured output when requested to provide comprehensive financial reports."""

# 以前の設定を継続
bedrock_model = BedrockModel(
    model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    region_name="us-west-2",
    temperature=0.0,  # 財務アドバイスのための決定論的な応答
)


@tool
def calculate_budget(monthly_income: float) -> str:
    """指定された月収に対して 50/30/20 予算内訳を計算します。"""
    needs = monthly_income * 0.50
    wants = monthly_income * 0.30
    savings = monthly_income * 0.20
    return f"💰 ${monthly_income:,.0f}/月の予算:\n• 必需品: ${needs:,.0f} (50%)\n• 欲しいもの: ${wants:,.0f} (30%)\n• 貯蓄: ${savings:,.0f} (20%)"


@tool
def create_financial_chart(
    data_dict: dict, chart_title: str = "Financial Chart"
) -> str:
    """財務データの辞書から円グラフの可視化を作成します。"""
    if not data_dict:
        return "❌ チャート用のデータが提供されていません"

    labels = list(data_dict.keys())
    values = list(data_dict.values())
    colors = ["#FF6B6B", "#4ECDC4", "#45B7D1", "#96CEB4", "#FECA57", "#FF9FF3"]

    plt.figure(figsize=(8, 6))
    plt.pie(
        values,
        labels=labels,
        autopct="%1.1f%%",
        colors=colors[: len(values)],
        startangle=90,
    )
    plt.title(f"📊 {chart_title}", fontsize=14, fontweight="bold")
    plt.axis("equal")
    plt.tight_layout()
    plt.show()

    return f"✅ {chart_title} の可視化を作成しました！"


# 完全な財務エージェントを作成
budget_agent = Agent(
    model=bedrock_model,
    system_prompt=BUDGET_SYSTEM_PROMPT,
    tools=[calculate_budget, create_financial_chart, calculator],
    callback_handler=None,
)

if __name__ == "__main__":
    # structured_output を使用して構造化出力をテスト
    print("\n構造化された財務レポート:")
    structured_response = budget_agent.structured_output(
        output_model=FinancialReport,
        prompt="月収6000ドルで外食に800ドル使っている人の包括的な財務レポートを生成してください。",
    )
    print(f"収入: ${structured_response.monthly_income:,.0f}")
    for category in structured_response.budget_categories:
        print(
            f"• {category.name}: ${category.amount:,.0f} ({category.percentage:.1f}%)"
        )
    print(f"\n財務健全性スコア: {structured_response.financial_health_score}/10")
    print("\n推奨事項:")
    for i, rec in enumerate(structured_response.recommendations, 1):
        print(f"{i}. {rec}")

In [ ]:
# エクスポートした予算エージェント実装をテスト
!python budget_agent.py 